# Preparing data frames for machine learning methods

This iterates through each data frame made in the previous pipe aned performs:

a) KNN to fill in missing "Number of ticket_validations" 

b) Converts the niehborhood classification with one hot encoding

c) identifies outlier unvalidated ticket counts and shrinks it relative to how much of an outlier it is, and the degree of support it has

In [5]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd 
import numpy as np
import glob
import os 

### Inputing 

In [2]:
# WARNING - This imputation is likely incorrect. It would be wise to invesitgate WHY the value is NAN to begin with 
# it could be because there simlpy was not ticket validations for that time!!!
# However, KNN should still provide a very low number if that is the case 
# However, it is something to be further inverstigated in the future

def knn_fill_validation(df, k=10):
    enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    enc.fit(df[["NIMI"]])

    df_sub = df.copy()

    # One-hot encode NIMI with global encoder
    nimi_encoded = enc.transform(df_sub[["NIMI"]])
    nimi_cols = enc.get_feature_names_out(["NIMI"])
    nimi_df = pd.DataFrame(nimi_encoded, columns=nimi_cols, index=df_sub.index)

    # Build numeric matrix
    matrix = pd.concat([
        nimi_df,
        df_sub[["Pre_2019", "density", "Rel_Visiting_score", "Validation_Count_mean"]]
    ], axis=1)

    # Run imputer
    imputer = KNNImputer(n_neighbors=k)
    imputed = imputer.fit_transform(matrix)

    # Put imputed values back
    df_sub["Validation_Count_mean"] = imputed[:, -1]

    return df_sub
    

def Encoding_variables(df):
    # extracting out time features 
    Final = df.drop(columns = "Stop_ID")
    Final["Time_min"] = (Final["Binned_time"].dt.hour)*60
    
    Final["time_sin"] = np.sin(2*np.pi*Final["Time_min"]/1440)
    Final["time_cos"] = np.cos(2*np.pi*Final["Time_min"]/1440)
    
    Final.drop(columns = ["Time_min"], inplace = True)


    # one hot encoding day, inlcuding day of the week (if it exists)    

    # Drop Binned_day if it only contains 0 
    if len(df["Binned_day"].unique() > 1): 
        Final = pd.get_dummies(Final, columns = ["NIMI", "Binned_day"])

    else:
        Final = pd.get_dummies(Final, columns = ["NIMI"])
        Final.drop(columns = ["Binned_day"], inplace = True)
        

    Final.drop(columns = ["Binned_time"], inplace = True)
    return Final
    





### One hot encoding

In [3]:
def Encoding_variables(df):
    # extracting out time features 
    Final = df.drop(columns = "Stop_ID")
    Final["Time_min"] = (Final["Binned_time"].dt.hour)*60
    
    Final["time_sin"] = np.sin(2*np.pi*Final["Time_min"]/1440)
    Final["time_cos"] = np.cos(2*np.pi*Final["Time_min"]/1440)
    
    Final.drop(columns = ["Time_min"], inplace = True)


    # one hot encoding day, inlcuding day of the week (if it exists)    

    # Drop Binned_day if it only contains 0 
    if len(df["Binned_day"].unique() > 1): 
        Final = pd.get_dummies(Final, columns = ["NIMI", "Binned_day"])

    else:
        Final = pd.get_dummies(Final, columns = ["NIMI"])
        Final.drop(columns = ["Binned_day"], inplace = True)
        

    Final.drop(columns = ["Binned_time"], inplace = True)
    return Final

### Outlier shrinking

In [4]:
def Outlier_shrinking(df, col="Unvalidated_mean", support_col="n_rows"):
    min_support = 3
    threshold = df["Unvalidated_mean"].quantile(0.95)

    # Calculate shrinkage factor
    def shrink(row):
        if row[col] > threshold and row[support_col] < min_support:
            # Proportional shrinkage: the smaller the support, the more we shrink
            support_factor = row[support_col] / min_support  # 0-1
            extreme_factor = threshold / row[col]           # 0-1
            shrink_factor = support_factor * extreme_factor
            return row[col] * shrink_factor
        else:
            return row[col]
    
    df[col] = df.apply(shrink, axis=1)
    return df

In [6]:

Performance = []

files = glob.glob("c:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/2_Granularity_tuning/Pipe1/*.csv")
output_folder = "C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/2_Granularity_tuning/Pipe2"
for file in files:
    Combo = os.path.basename(file).split(".")[0]
    print(Combo)
    df = pd.read_csv(file, parse_dates = ["Binned_time"])
    Imputed_missing = knn_fill_validation(df)
    # Drop any remaining NA values 
    Imputed_missing.dropna(inplace = True)

    # Encode variables 
    Encoded = Encoding_variables(Imputed_missing)

    # Shrinking outliers 
    Shrunk = Outlier_shrinking(Encoded)

    # export df 
    Name = f"{Combo}_Final.csv"
    Shrunk.to_csv(f"{output_folder}/{Name}")
    

Combo_120min_Collapseall
Combo_120min_Dayofweek
Combo_120min_Weekendsplit
Combo_180min_Collapseall
Combo_180min_Dayofweek
Combo_180min_Weekendsplit
Combo_30min_Collapseall
Combo_30min_Dayofweek
Combo_30min_Weekendsplit
Combo_360min_Collapseall
Combo_360min_Dayofweek
Combo_360min_Weekendsplit
Combo_60min_Collapseall
Combo_60min_Dayofweek
Combo_60min_Weekendsplit
Combo_720min_Collapseall
Combo_720min_Dayofweek
Combo_720min_Weekendsplit
